In [1]:
from collections import defaultdict
import sqlalchemy as sa
import sqlalchemy.orm as so
import sqlalchemy.sql.sqltypes as sss
import sqlalchemy.sql.expression as exp
import re

from omop_alchemy.db import Base, engine
from omop_alchemy.tables.vocabulary import Concept, Concept_Relationship, Concept_Ancestor

In [2]:
import enum

class ConceptEnum(enum.Enum):

    @classmethod
    def member_values(cls):
        return (s.value for s in cls)
    
    @classmethod
    def is_member(cls, val):
        return not val or val in [s.value for s in cls]

    @classmethod
    def labels(cls):
        return [s.name for s in cls]
    

class TNM(ConceptEnum):
    parent_mod = 734320

In [35]:

class VocabLookup:
    # base class for custom vocabulary lookups

    # correction parameter holds an ordered list of callable corrections 
    # - try match the raw input string first 
    # - then apply corrections in order and return the first match 
    # - examples of correction functions would be stripping punctuation, 
    #   spelling correction functions

    def __init__(self, 
                 unknown=0,           # TODO: tbd do we want to define behaviours when mapping is not found?
                 parent=None,         # used when you want to pull all child concepts under a given parent into the lookup
                 domain=None,         # otherwise we are grabbing by specification of domain
                 concept_class=None,  # and optionally concept_class
                 standard_only=True): # for when you want to toggle between grabbing children from standard concepts strictly or not
        self._unknown = unknown.value if isinstance(unknown, ConceptEnum) else unknown
        self._lookup = defaultdict(self.return_unknown)
        self._domain = domain
        self._concept_class = concept_class
        self._standard_only = standard_only
        # parent parameter is the high-level concept under which you want to pull
        # in all available matches - e.g. TNM stages, which can grab all concepts 
        # that fall under the parent concept from the concept_relationship table
        self._parent = parent.value if isinstance(parent, ConceptEnum) else parent
        self._correction = None
        with so.Session(engine) as session:
            # TBD: question - do we need to provide support for combining parent 
            # definition with domain def? is this a likely use-case? it won't fail 
            # for now, but perhaps check?
            if parent is not None:
                self.get_lookup(session)
            if domain is not None:
                self.get_domain_lookup(session)
        
        # TODO: consider generalisable creation of custom maps to host 
        # manual mappings of local concepts to OMOP concepts as well?

    
    def get_domain_lookup(self, session):
        # returns a default dictionary that contains all
        # concepts under a given domain for rapid lookups
        
        q = session.query(Concept)
        if self._domain:
            q = q.filter(Concept.domain_id==self._domain)
        if self._concept_class:
            q = q.filter(Concept.concept_class_id==self._concept_class)
        if self._standard_only:
            q = q.filter(Concept.standard_concept=='S')
        concepts = q.all()
        for row in concepts:
            self._lookup[row.concept_name.lower().strip()] = row.concept_id
            self._lookup[row.concept_code.lower().strip()] = row.concept_id
    
    def get_standard_hierarchy(self, session):
        children = session.query(Concept_Ancestor
                                ).options(so.joinedload(Concept_Ancestor.descendant)
                                ).filter(Concept_Ancestor.ancestor_concept_id == self._parent).distinct().all()
        return [c.descendant for c in children]
        
    def get_all_hierarchy(self, this_level, concepts, session):
        # TODO: check if we want to do this thru Concept_Ancestor strictly
        # if confirmed we only want to be doing for standard concepts?
        # this is iterative and slow way of doing it to arbitrary depths
        # otherwise...but good if you want to include non-standard 
        # children - maybe useful in condition_concept_id?

        if len(this_level) == 0:
            return concepts
        children = session.query(Concept
                                ).join(Concept_Relationship, Concept_Relationship.concept_id_2==Concept.concept_id
                                ).filter(Concept_Relationship.concept_id_1.in_(this_level)
                                ).filter(Concept_Relationship.relationship_id=='Subsumes').distinct().all()
        next_level = tuple([c.concept_id for c in children if c not in concepts])
        concepts += children
        concepts = self.get_all_hierarchy(next_level, concepts, session)
        return concepts

    def get_lookup(self, session):
        # returns a default dictionary that contains all
        # concepts under a given parent concept and the
        # appropriate unknown value for the target context
        if not self._standard_only:
            concepts = self.get_all_hierarchy(tuple([self._parent]), [], session)
        else:
            concepts = self.get_standard_hierarchy(session)
        for c in concepts:
            self._lookup[c.concept_name.lower()] = c.concept_id
            self._lookup[c.concept_code.lower()] = c.concept_id

    def return_unknown(self):
        return self._unknown

    def lookup_exact(self, term):
        if term == None:
            term = ''
        return self._lookup[term.lower().strip()]

    def lookup(self, term):
        if term == None:
            term = ''
        value = self._lookup[term.lower().strip()]
        if self._correction is not None:
            for c in self._correction:
                if value != self._unknown:
                    break
                value = self._lookup[c(term).lower().strip()]
        return value

In [36]:
tnm = VocabLookup(parent=TNM.parent_mod)

In [37]:
tnm.lookup('ajcc/uicc 8th pathological t1a category'), tnm.lookup('AJCC/UICC 7th Pathological t1a Category'), tnm.lookup('AJCC/UICC pathological Stage 3A1'), tnm.lookup('AJCC/UICC Stage 3A1')

(1633374, 1633268, 1633303, 1635412)

In [38]:
measurements = VocabLookup(domain='Measurement', concept_class='Extension/Invasion')

In [40]:
measurements.lookup('initial diagnosis')

734306

In [26]:
measurements._concept_class, measurements._domain, measurements._standard_only

('Extension/Invasion', 'Measurement', True)